<a href="https://colab.research.google.com/github/AlokDhanush/RAG_chatbot/blob/main/RCB_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q faiss-cpu sentence-transformers transformers accelerate bitsandbytes

In [ ]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

In [ ]:
file_path = "/rcb_data.txt"
with open(file_path, "r", encoding="utf-8") as f:
    text = f.read()

In [ ]:
def chunk_text(text, max_words=100):
    words = text.split()
    return [' '.join(words[i:i+max_words]) for i in range(0, len(words), max_words)]

chunks = chunk_text(text)

In [ ]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedder.encode(chunks, convert_to_numpy=True).astype("float32")

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_id = "meta-llama/Llama-3.2-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16
)

llm = pipeline("text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [ ]:
def get_top_chunks(query, k=3):
    query_vec = embedder.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_vec, k)
    return [chunks[i] for i in indices[0]]

def build_prompt(context_chunks, query):
    context = "\n".join(context_chunks)
    prompt = f"""[INST] You are a cricket expert assistant. Use only the context to answer.
If the answer is not found in the context, say "I don't know".

### Context:
{context}

### Question:
{query}

### Answer:
[/INST]"""
    return prompt

In [ ]:
def ask(query):
    context_chunks = get_top_chunks(query)
    prompt = build_prompt(context_chunks, query)

    result = llm(
        prompt,
        max_new_tokens=200,
        temperature=0.3,
        do_sample=False,
        return_full_text=False
    )

    return "Answer: "+ result[0]['generated_text'].strip()


In [ ]:
query = "Who has the most runs in IPL?"
print("Retrieved Context:\n", "\n---\n".join(get_top_chunks(query)))
ask(query)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


🔍 Retrieved Context:
 68 in their first encounter, putting them under pressure because of their negative run rate through their otherwise strong campaign.[165] In the latest chapter of the rivalry between the two in IPL 2024, like the Kolkata Knight Riders, Sunrisers Hyderabad were the first to break the 263-run record set by RCB, which seemed insurmountable at the time as a result of Chris Gayle's 175. In a more humiliating turn, SRH again broke RCB's record against RCB themselves on their home ground, scoring 287 runs, thanks to a 39-ball century by Travis Head and a 30-ball 67 by Heinrich Klassen. Although
---
as head coach.[123] RCB qualified for the playoffs by finishing third in the points table but failed to lift the trophy once again, losing to fourth placed Kolkata Knight Riders in the Eliminator.[124][125] Harshal Patel finished the season with 32 wickets, equalling Dwayne Bravo's record for the highest number of wickets taken in a season and winning the Purple Cap.[126][127]

'Answer: Glenn Maxwell with 513 runs.'

In [ ]:
ask("How many runs did Virat Kohli scored in the final match between RCB vs PBKS?")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"Answer: I don't know. The context only mentions Virat Kohli scoring 75 runs in one match, but it does not specify the opponent or the final match."

In [ ]:
ask("How many times has RCB played playoff matches?")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"Answer: I don't know."

In [ ]:
ask("Who is the owner of RCB?")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"Answer: I don't know. The context does not mention the owner of RCB."

In [ ]:
ask("RCB stands for?")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"Answer: Royal Challengers Bangalore, commonly known as RCB, is a professional T20 franchise cricket team based in Bengaluru, Karnataka, that competes in the Indian Premier League. Founded in 2008 by United Spirits, the team's home ground is M. Chinnaswamy Stadium. RCB won their first title in 2025.[4] The team has also finished as the runners-up on three occasions: in 2009, 2011, and 2016. They have also qualified for the playoffs in ten of the eighteen seasons. As of 2025, the team is captained by Rajat Patidar and coached by Andy Flower. The franchise has competed redesigned in 2016 with the inclusion of black as a secondary colour. The lion emblem in the crest was enlarged and the shield was omitted in the new design. In 2020, a new logo was unveiled featuring a bigger lion and the crown returning from the previous logo. The RC emblem was omitted"

In [ ]:
ask("When did RCB win their first title?")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'Answer: RCB won their first title in 2025.'

In [ ]:
ask("Who is the top scorer for RCB in 2025?")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"Answer: I don't know. The context only provides information about the 2023 season, not the 2025 season."

In [ ]:
ask("Ok, who is the top scorer for RCB?")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'Answer: Glenn Maxwell with 513 runs.'